# Training Model From Pre-trained Model

## Dataset

Dataset was cleaned and uploaded to huggingface at `thisisfrantz/haitian-creole-english-train` for the train set and `thisisfrantz/haitian-creole-english-test` for the test set. 

"koman _" -> "koman ou ye"

In [1]:
from datasets import load_dataset

dataset = load_dataset("thisisfrantz/haitian-creole-english-train")

c:\Users\fpaul\anaconda3\envs\nlpENV\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\fpaul\anaconda3\envs\nlpENV\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\fpaul\.cache\huggingface\hub\datasets--thisisfrantz--haitian-creole-english-train. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. I

In [2]:
print(dataset)
print(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['id', 'lang1', 'lang2'],
        num_rows: 10813
    })
})
{'id': 3042, 'lang1': 'Lidè Kiben an te di, ata John Kennedy dwe cheche fason pou kontoune anbago a.', 'lang2': 'Even John F. Kennedy had to find a way around the embargo, the Cuban leader said.'}


## Load Pretrained Tokenizer

I wanted to create a custom tokenizer but don't have enough data :( .

In [4]:
from transformers import MarianTokenizer

tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ht-en")

In [7]:

example = dataset['train'][0]

source_text = example['lang1']
target_text = example['lang2']
print("Source text:", source_text)
print("Target text:", target_text)

source_tokens = tokenizer(source_text)
source_ids = tokenizer.convert_tokens_to_ids(source_tokens)

# Tokenize target (as target tokenizer)
with tokenizer.as_target_tokenizer():
    target_tokens = tokenizer.tokenize(target_text)
    target_ids = tokenizer.convert_tokens_to_ids(target_tokens)

print("\nSource Tokens:", source_tokens)
print("Source Token IDs:", source_ids)

print("\nTarget Tokens:", target_tokens)
print("Target Token IDs:", target_ids)

Source text: Lidè Kiben an te di, ata John Kennedy dwe cheche fason pou kontoune anbago a.
Target text: Even John F. Kennedy had to find a way around the embargo, the Cuban leader said.

Source Tokens: {'input_ids': [116, 16401, 61, 14693, 32, 7, 48, 2, 8611, 424, 29389, 9732, 108, 20212, 113, 14, 300, 2556, 4424, 276, 5887, 8, 3, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
Source Token IDs: [1, 1]

Target Tokens: ['▁Even', '▁John', '▁F', '.', '▁Kenne', 'dy', '▁had', '▁to', '▁find', '▁a', '▁way', '▁around', '▁the', '▁emb', 'ar', 'go', ',', '▁the', '▁Cuba', 'n', '▁leader', '▁said', '.']
Target Token IDs: [871, 424, 1316, 3, 29389, 9732, 129, 10, 504, 8, 222, 1293, 6, 30292, 4423, 5887, 2, 6, 22471, 430, 5026, 260, 3]


c:\Users\fpaul\anaconda3\envs\nlpENV\lib\site-packages\transformers\tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
# Tokenize the whole dataset
def tokenize_function(example):
    inputs = tokenizer(example['lang1'], truncation=True, padding='max_length', max_length=128)
    with tokenizer.as_target_tokenizer():
        targets = tokenizer(example['lang2'], truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = targets['input_ids']
    return inputs

# preprocess the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/10813 [00:00<?, ? examples/s]c:\Users\fpaul\anaconda3\envs\nlpENV\lib\site-packages\transformers\tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 10813/10813 [00:04<00:00, 2217.35 examples/s]


## DataLoader

In [17]:
from torch.utils.data import DataLoader

# PyTorch Format
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Create DataLoader
train_loader = DataLoader(tokenized_dataset['train'], batch_size=8, shuffle=True)

## Full Training Loop

In [12]:

def create_dataloaders(dataset_name, batch_size=8):
    train_dataset = load_dataset(dataset_name + '-train', split='train')
    test_dataset = load_dataset(dataset_name + '-test', split='validation')

    # Tokenize the datasets
    train_dataset = train_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)

    # Set the format for PyTorch
    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    return train_dataloader, test_dataloader

In [ ]:
from torch.optim import AdamW

# Load tokenizer & model
model_name = "Helsinki-NLP/opus-mt-ht-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# dataset and autoloader
train_dataloader, test_dataloader = create_dataloaders("thisisfrantz/haitian-creole-english")

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)